In [1]:
import torch
from torch import nn

import cheetah

In [2]:
quadrupole = cheetah.Quadrupole(length=torch.tensor(0.2), k1=torch.tensor(4.2))
quadrupole

Quadrupole(length=0.20, k1=tensor(4.2000), misalignment=tensor([0., 0.]), tilt=0.00, name="None", device="cpu")

In [3]:
quadrupole.k1

tensor(4.2000)

In [4]:
list(quadrupole.parameters())

[]

In [5]:
quadrupole.k1 = nn.Parameter(torch.tensor(5.0))
quadrupole

Quadrupole(length=0.20, k1=Parameter containing:
tensor(5., requires_grad=True), misalignment=tensor([0., 0.]), tilt=0.00, name="None", device="cpu")

In [6]:
list(quadrupole.parameters())

[Parameter containing:
 tensor(5., requires_grad=True)]

In [7]:
segment = cheetah.Segment(elements=[quadrupole])
segment

Segment([Quadrupole(length=0.20, k1=Parameter containing:
tensor(5., requires_grad=True), misalignment=tensor([0., 0.]), tilt=0.00, name="None", device="cpu")])

In [8]:
list(segment.parameters())

[Parameter containing:
 tensor(5., requires_grad=True)]

In [13]:
type(segment.elements[0].k1)

torch.nn.parameter.Parameter

In [9]:
class TestModule(nn.Module):
    """Test"""

    def __init__(self, k1: torch.Tensor = torch.tensor(0.0)) -> None:
        super().__init__()

        self.k1 = k1


test_module = TestModule(k1=torch.tensor(4.2))
list(test_module.parameters())

[]

In [10]:
test_module.k1 = nn.Parameter(torch.tensor(3.14))
list(test_module.parameters())

[Parameter containing:
 tensor(3.1400, requires_grad=True)]

In [14]:
s = "hello"
print(s)
print(repr(s))

hello
'hello'
